In [ ]:
import zipfile

zip_file_name = "tuned2.zip" # image 50张图像; image1 600张图像; image2 650张图像; test 13张图像

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall()

# 后处理操作

In [ ]:
import os
import cv2

# 设置文件夹路径
input_folder_path = 'tuned2/'
output_folder_path = 'mask02/'

# 获取文件夹中所有图像文件
image_files = [f for f in os.listdir(input_folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for image_file in image_files:
    # 读取图像
    image_path = os.path.join(input_folder_path, image_file)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 以灰度图像格式读取图像

    # 找到连通组件
    ret, labels = cv2.connectedComponents(image)

    # 去除小的连通区域
    min_area = 250  # 最小连通区域面积
    for label in range(1, ret):
        area = np.sum(labels == label)
        if area < min_area:
            image[labels == label] = 0

    # 保存处理后的图像到输出文件夹
    output_path = os.path.join(output_folder_path, image_file)
    cv2.imwrite(output_path, image)

print("图像处理完成并保存到 mask02 文件夹中。")

图像处理完成并保存到 mask02 文件夹中。


In [ ]:
import os
import numpy as np
import cv2
from sklearn.metrics import average_precision_score

def binarize_masks(masks, threshold=0):
    binarized_masks = [mask > threshold for mask in masks]
    return binarized_masks

def calculate_iou(pred_mask, true_mask):
    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()
    iou = intersection / union if union > 0 else 0.0
    return iou

# Load predicted masks from mask0 folder
predicted_mask_folder = 'mask02/' #mask1 orig_new combined_masks1/ tuned_image_new/
predicted_mask_files = os.listdir(predicted_mask_folder)
predicted_masks = [cv2.imread(os.path.join(predicted_mask_folder, file), cv2.IMREAD_GRAYSCALE) for file in predicted_mask_files]
predicted_masks = binarize_masks(predicted_masks)

# Load true masks from mask1 folder
true_mask_folder = 'test_mask/'
true_mask_files = os.listdir(true_mask_folder)
true_masks = [cv2.imread(os.path.join(true_mask_folder, file), cv2.IMREAD_GRAYSCALE) for file in true_mask_files]
true_masks = binarize_masks(true_masks)

# Initialize metrics
num_classes = 2  # Assuming binary masks
iou_scores = []
dices = []
precisions = []
recalls = []
f1_scores = []
average_precisions = []

# Calculate metrics
for class_idx in range(num_classes):
    ious = []
    dices = []
    class_true_positive = 0
    class_false_positive = 0
    class_false_negative = 0

    for pred_mask, true_mask in zip(predicted_masks, true_masks):
        iou = calculate_iou(pred_mask, true_mask)
        ious.append(iou)

        dice = 2 * np.logical_and(pred_mask, true_mask).sum() / (pred_mask.sum() + true_mask.sum() + 1e-5)
        dices.append(dice)

        true_positive = np.logical_and(pred_mask, true_mask).sum()
        false_positive = np.logical_and(pred_mask, ~true_mask).sum()
        false_negative = np.logical_and(~pred_mask, true_mask).sum()

        class_true_positive += true_positive
        class_false_positive += false_positive
        class_false_negative += false_negative

    class_precision = class_true_positive / (class_true_positive + class_false_positive)
    class_recall = class_true_positive / (class_true_positive + class_false_negative)
    class_f1 = 2 * (class_precision * class_recall) / (class_precision + class_recall)

    iou_scores.append(sum(ious) / len(ious))
    precisions.append(class_precision)
    recalls.append(class_recall)
    f1_scores.append(class_f1)

    # Calculate Average Precision (AP) for each class
    true_masks_stacked = np.stack(true_masks, axis=-1)
    predicted_masks_stacked = np.stack(predicted_masks, axis=-1)
    for class_idx in range(num_classes):
        true_class = true_masks_stacked[:, :, class_idx].flatten()
        predicted_class = predicted_masks_stacked[:, :, class_idx].flatten()
        average_precision = average_precision_score(true_class, predicted_class)
        average_precisions.append(average_precision)

# Calculate mean metrics
mean_iou = sum(iou_scores) / num_classes
mean_dice = np.mean(dices)
mean_precision = sum(precisions) / num_classes
mean_recall = sum(recalls) / num_classes
mean_f1 = sum(f1_scores) / num_classes
mean_average_precision = sum(average_precisions) / num_classes

# Print evaluation metrics
print(f"Mean Intersection over Union (mIoU): {mean_iou}")
print(f"Mean Dice Coefficient: {mean_dice}")
print(f"Mean Precision: {mean_precision}")
print(f"Mean Recall: {mean_recall}")
print(f"Mean F1 Score: {mean_f1}")
print(f"Mean Average Precision (mAP): {mean_average_precision}")

Mean Intersection over Union (mIoU): 0.07238309943551793
Mean Dice Coefficient: 0.1295016583489367
Mean Precision: 0.1338066130817731
Mean Recall: 0.1561643373927111
Mean F1 Score: 0.14412354780012132
Mean Average Precision (mAP): 0.10645907434962812


In [ ]:
import shutil

# 指定要下载的文件夹路径
mask_folder_path = "mask02/"  # 根据你实际的文件夹路径进行修改

# 指定zip文件名
zip_file_name = "mask02.zip"

# 将文件夹打包成zip文件
shutil.make_archive("mask02", 'zip', mask_folder_path)

'/content/mask02.zip'

In [ ]:
from google.colab import files

# 下载zip文件
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>